### Use this markdown to generate the predictors


In [2]:
#imported libraries
import pandas as pd
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import langdetect
import datetime
%matplotlib inline  
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaseWeightBoosting
from sklearn.preprocessing import Imputer

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#Creating of the input data
downloads = pd.read_csv('train_app_downloads.csv')
reviews = pd.read_csv('train_app_review.csv')
ratings = pd.read_csv('train_app_rating.csv')
usages = pd.read_csv('train_usage.csv')
revenues = pd.read_csv('train_revenue.csv')
output = pd.read_csv('train_final_downloads.csv')
prev_downloads = pd.read_csv('train_cumulative_downloads_2015-02.csv').drop('Unnamed: 0', 1)
dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))



text_score = pd.read_csv('sentiment.csv',header=-1).ix[:,0]
title_score = pd.read_csv('t_sentiment.csv',header=-1).ix[:,0]
sentiment = pd.Series([0.5]*title_score.shape[0])
for i in range(title_score.shape[0]):
    if title_score.values[i] == 0:
        if text_score.values[i]>0: sentiment.values[i] = text_score.values[i]
    else:
        if text_score.values[i]>0: sentiment.values[i] = (text_score.values[i]+title_score.values[i])/2
        else: sentiment.values[i] = title_score.values[i]
reviews["sentiment_score"] = sentiment.values 

#reviews = pd.read_csv('reviews_modified.csv').drop('Unnamed: 0', 1)

#We map -1 to 0 in the downloads (there are no 0 in the initial data)
replacementValue=0
downloads = downloads.replace(-1,replacementValue)

'''imputer = Imputer(missing_values=-1, strategy='median', axis=0)
category = list(set(downloads["category"].values))
imp = pd.DataFrame(columns = downloads.columns)
for cat in category:
    for dev in ["iphone","ipad"]:
        curr_df = downloads.ix[downloads["category"]==cat,:]
        curr_df = curr_df.ix[curr_df["device"]==dev,:]
        name = curr_df.columns
        df1 = curr_df.ix[:,0:5]
        df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,5:]))
        df2.index = df1.index
        curr_df = pd.concat([df1,df2],axis = 1)
        curr_df.columns = name 
        imp = pd.concat([imp,curr_df],axis = 0)
downloads = imp


imputer = Imputer(missing_values=-1, strategy='median', axis=0)
category = list(set(usages["category"].values))
imp = pd.DataFrame(columns = usages.columns)
for cat in category:
    #for dev in ["iphone","ipad"]:
        for metric in range(1,5):
            curr_df = usages.ix[usages["category"]==cat,:]
            #curr_df = curr_df.ix[curr_df["device"]==dev,:]
            curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            name = curr_df.columns
            df1 = curr_df.ix[:,0:6]
            df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,6:]))
            df2.index = df1.index
            curr_df = pd.concat([df1,df2],axis = 1)
            curr_df.columns = name 
            imp = pd.concat([imp,curr_df],axis = 0)
usages = imp


#name = revenues.columns
#revenues = pd.concat([revenues.ix[:,0:5],pd.DataFrame(imputer.fit_transform(revenues.ix[:,5:]))],axis = 1)
#revenues.columns = name 

imputer = Imputer(missing_values=-1, strategy='median', axis=0)
category = list(set(revenues["category"].values))
imp = pd.DataFrame(columns = revenues.columns)
for cat in category:
    for dev in ["iphone","ipad"]:
        #for metric in range(1,5):
            curr_df = revenues.ix[revenues["category"]==cat,:]
            curr_df = curr_df.ix[curr_df["device"]==dev,:]
            #curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            name = curr_df.columns
            df1 = curr_df.ix[:,0:5]
            df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,5:]))
            df2.index = df1.index
            curr_df = pd.concat([df1,df2],axis = 1)
            curr_df.columns = name 
            imp = pd.concat([imp,curr_df],axis = 0)
revenues = imp

'''

#Minor corrections
ratings = ratings.rename(columns={'start1': 'star1'})
ratings = pd.merge(downloads.drop(dateRange,1), ratings.drop('Unnamed: 0', 1), how='left',
                   on=["id","name","category"]).replace("NaN",replacementValue)

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


ImportError: cannot import name BaseWeightBoosting

#### Initialize the predictors matrix

In [ ]:
predictors = pd.concat([downloads["id"],downloads["name"],downloads["category"],downloads["device"]],
                       axis=1,keys=["id","name","category","device"])
predictors.head()

#### Use the train_app_downloads.csv file

In [ ]:
#Generate the logWeekly average
def generator_weekAvg(inp,w):
    if (np.count_nonzero(inp[5+w*7:12+w*7] - replacementValue*np.ones(len(inp[5+w*7:12+w*7]))) == 0):
        return 0
    #return  math.log(1.0*sum(inp[5+w*7:12+w*7])/np.count_nonzero(inp[5+w*7:12+w*7] 
    #                                                             - replacementValue*np.ones(len(inp[5+w*7:12+w*7]))))
    return  1.0*sum([np.log(c) for c in inp[5+w*7:12+w*7] if c > 0])/np.count_nonzero(inp[5+w*7:12+w*7] 
                                                                 - replacementValue*np.ones(len(inp[5+w*7:12+w*7])))
for w in range(8):
    predictors["week_"+str(w+1)] = downloads.apply(generator_weekAvg,axis=1,args=(w,))

In [ ]:
#Generate the daily average
def generator_dailyAvg(inp):
    if (np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))) == 0):
        return 0
    #return  (1.0*sum(inp[5:])/np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))))
    #return  math.log(1.0*sum(inp[5:])/len(inp[5:]))
    return  (1.0*sum([np.log(c) for c in inp[5:] if c>0 ])/np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))))
    
predictors["daily_avg"] = downloads.apply(generator_dailyAvg,axis=1)
predictors["download_sum"] = downloads.ix[:,5:].sum(axis=1)
#This one is bad

In [ ]:
#Generate the polynomial coefficients
def generator_coef(inp, coef):
    #return  np.polyfit(range(56),np.cumsum([np.log(c) for c in inp[5:] if c>0]),coef)[0]
    
    Y = [np.log(c) for c in inp[5:] if c>0]
    if len(Y)<=1: return 0
    return  np.polyfit(range(len(Y)),np.cumsum(Y),coef)[0]
#Redo by ignoring the -1 / 0 ? w = [1110011]     w = np.not_equal(inp[5:],np.ones(len(inp[5:]))*replacementValue

for c in range(4):
    predictors["coef_"+str(c)] = downloads.apply(generator_coef,axis=1,args=(c,))

In [ ]:
#Generate the step max and min (we ignore the values of 0)
def generator_maxStep(inp,maximum):
    if (np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))) == 0):
        return 0
    m = 0
    for d in range(1,56):
        #if (inp[5+d]!=replacementValue and inp[4+d]!=replacementValue):
        if (inp[5+d]>0 and inp[4+d]>0):
            #c = (inp[5+d]-inp[4+d])
            c = float(inp[5+d])/inp[4+d]
            if (maximum and m < c):
                m = c
            if ( not maximum and m > c):
                m = c
    #return m
    if m==0: return 0
    return np.log(m)
                
predictors["maxStep"] = downloads.apply(generator_maxStep,axis=1,args=(True,))
predictors["minStep"] = downloads.apply(generator_maxStep,axis=1,args=(False,))

#Standard Deviation
def generator_std(inp):
    if (np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))) == 0):
        return 0
    return np.std(inp[5:])
    #Y = [np.log(c) for c in list(inp[5:].values) if c > 0]
    #if len(Y) == 0 : return 0
    #return np.std(Y)

predictors["std"] = downloads.apply(generator_std,axis = 1)

#Number of missing Values
def generator_missing(inp):
    return list(inp[5:]).count(replacementValue)
    #return np.count_zero(inp[5:] -replacementValue*np.ones(len(inp[5:])))

predictors["nb_missing"] = downloads.apply(generator_missing,axis = 1)

In [ ]:
#device
def generator_iphone(inp):
    if (inp[4] == "iphone"):
        return 1
    else:
        return 0

def generator_ipad(inp):
    if(inp[4] == "ipad"):
        return 1
    else:
        return 0
    
predictors["iphone"] = downloads.apply(generator_iphone,axis = 1)
#predictors["ipad"] = downloads.apply(generator_ipad,axis = 1)

In [ ]:
#Categories
def generator_categories(inp,cat):
    if (inp[3] == cat):
        return 1
    else:
        return 0
    

for cat in list(set(downloads["category"]))[:-1]:
    predictors[cat] = downloads.apply(generator_categories,axis = 1,args=(cat,))

In [ ]:
def detect_language(x):
    try:
        detected = langdetect.detect(x.decode('utf8','ignore'))
        if detected in ['ja']:
            return 'japanese'
        elif detected in ['zh-cn']:
            return 'chinese'
        elif detected in ['ko']:
            return 'korean'
        elif detected in ['en']:
            return 'english'
        return 'other'
    except:
        return 'other'
        #return None

def set_lang_categories(x, cat):
    if x == cat:
        return 1
    else:
        return 0
    
lang_series = predictors['name'].apply(detect_language)
for cat in list(set(lang_series)):
    predictors[cat] = lang_series.apply(set_lang_categories, args=(cat,))
    

#### Use the train_cumulative_downloads_2015-02.csv file

In [ ]:
#prev_downloads = pd.read_csv('train_cumulative_downloads_2015-02.csv').drop('Unnamed: 0', 1)
predictors = pd.merge(predictors, prev_downloads, how='left',
                  on=["id","device"])

In [ ]:
predictors['dl_projection'] = np.log(predictors['cumulative_downloads_2015-02'] + 7 * predictors['download_sum'])

#### Use the train_app_rating.csv file and reviews

In [ ]:
avg_reviews = reviews.groupby('id').agg('mean')
avg_reviews['rating']
predictors['avg_review'] = predictors.join(avg_reviews['rating'],on='id')['rating'].replace("NaN",0)


In [ ]:
avg_reviews = reviews.groupby('id').agg('var')
avg_reviews['rating']
predictors['var_review'] = predictors.join(avg_reviews['rating'],on='id')['rating'].replace("NaN",0)


In [ ]:
ratings['num_ratings'] = ratings.ix[:,['star1','star2','star3','star4','star5']].sum(axis=1)
#Scaling the ratings
for i in range(1,6):
    ratings['star'+str(i)]=ratings['star'+str(i)].divide(ratings['num_ratings']+1)

In [ ]:
predictors.columns

In [ ]:
#raw ratings
predictors = pd.merge(predictors, ratings.drop('Unnamed: 0', 1), how='left',
                   on=["id","name","category","device"]).replace("NaN",replacementValue)


In [ ]:
#Reviews per daily downloads
#predictors['reviewPerDailyDownloads'] = predictors['num_ratings'].divide(predictors['daily_avg']+1)
predictors['ratings_per_downloads'] = predictors['num_ratings'].divide(predictors['cumulative_downloads_2015-02']+1)
#Drop either this or num_ratings

In [ ]:
num_versions = reviews.groupby('id').version.nunique()
num_versions.name = 'num_versions'
predictors = predictors.join(num_versions, how='left', on='id')
predictors['num_versions'] = predictors['num_versions'].replace('NaN',0)

In [ ]:
num_review = reviews.groupby('id').rating.count()
num_review.name = 'num_review'
predictors = predictors.join(num_review, how='left', on='id')

predictors['review_per_downloads'] = predictors['num_review'].replace('NaN',0).divide(predictors['cumulative_downloads_2015-02']+1)

def gini_impurity(inp):
    tot = sum(inp[1:])
    if tot == 0:
        return 0.0
    return sum([1.0*x / tot * (1 - 1.0*x / tot) for x in inp[1:]])       *tot

sum_reviews = reviews.groupby('name')[list(set(reviews['language']))].sum().reset_index()
sum_reviews["gini_review"] = sum_reviews[list(set(reviews['language']))].apply(gini_impurity,axis=1)
predictors=predictors.merge(sum_reviews[["id","gini_review"]],on='id',how="left")

#### Use the train_release_date.csv file

In [ ]:
def generate_days_since_release(x):
    return (datetime.datetime.strptime('03/01/2015', '%m/%d/%Y').date() - datetime.datetime.strptime(x, '%m/%d/%Y').date()).days

release_date = pd.read_csv('train_release_date.csv').drop('Unnamed: 0', 1)
release_date['days_since_release'] = release_date['release_date'].apply(generate_days_since_release)
predictors = pd.merge(predictors, release_date.drop('release_date', 1), how='left',
                  on=["id","name"])

# replacing any missing values with 0; not a good idea but placeholder for now
predictors['days_since_release'] = predictors['days_since_release'].replace('NaN', 0)

# The number of downloads divided by time
predictors['downloads_per_day_before'] = predictors['cumulative_downloads_2015-02'].divide(predictors['days_since_release']+1)

In [ ]:

predictors['versions_per_day'] = predictors['num_versions'].divide(predictors['days_since_release']+60)
predictors['review_per_day'] = predictors['num_review'].divide(predictors['days_since_release']+60)
predictors['rating_per_day'] = predictors['num_ratings'].divide(predictors['days_since_release']+60)


#### Use the sentiment score

In [ ]:
#Later we can compute the weighted average of sentiment scores based on reviewers.
#add positive and negative columns to indicate the app's popularity

#print reviews.ix[id==predictors.id[0],:]["sentiment_score"]
#print reviews.ix[id==predictors.id[1],:]["sentiment_score"]

avg_score = [0.5]*predictors.shape[0]
predictors["positive"] = [0]*predictors.shape[0]
predictors["negative"] = [0]*predictors.shape[0]

reviewers = {}
for i in range(reviews.shape[0]):
    #if i%10000 ==0 : print i
    user = reviews['reviewer'].values[i]
    item_id = reviews['id'].values[i]
    if user in reviewers:
        reviewers[user] = reviewers[user] + predictors["week_8"][predictors.id==item_id].values.sum()
    else:
        reviewers[user] = predictors["week_8"][predictors.id==item_id].values.sum()
        
for i in range(predictors.shape[0]):
    #if i%10000 == 0: print i
    score_list = list(reviews.ix[reviews["id"]==predictors.id[i],:]['sentiment_score'].values)
    reviewer_list =  list(reviews.ix[reviews["id"]==predictors.id[i],:]['reviewer'].values)
    s = 0
    rs = 0 
    if len(score_list)==0: continue
    for j in range(len(score_list)):
        s = s + score_list[j]*reviewers[reviewer_list[j]]
        rs = rs + reviewers[reviewer_list[j]]
    avg_score[i] = s/rs
    #avg_score[i] = reviews.ix[reviews["id"]==predictors.id[i],:]["sentiment_score"].mean()
    if avg_score[i]>0.55: 
        predictors["positive"].values[i] = 1
    elif avg_score[i]<0.45: predictors["negative"].values[i] = 1

predictors["avg_sentiment_score"] = avg_score

 

In [ ]:
#np.savetxt("C:\\Users\\andre\\Desktop\\avg_score.csv",avg_score)

#### Use coeficients of metrics 

In [ ]:
for i in range(4):
    predictors["m"+str(i+1)+"_max"] = np.zeros(predictors.shape[0]) 
    #predictors["m"+str(i+1)+"_min"] = np.zeros(predictors.shape[0])
    predictors["m"+str(i+1)+"_mean"] = np.zeros(predictors.shape[0])
    #predictors["m"+str(i+1)+"_std"] = np.zeros(predictors.shape[0])
    for j in range(3):
        predictors["m"+str(i+1)+"_coef_"+str(j)] = np.zeros(predictors.shape[0])

for i in range(predictors.shape[0]):
    if predictors["id"].values[i] not in usages["id"].values: continue
    for j in range(4):
        tmp = usages.ix[usages["id"] == predictors["id"].values[i],:]
        time_series = np.array(tmp.ix[tmp["metric"] == j+1,6:14])[0]   
        if -1 in time_series: continue
        predictors["m"+str(j+1)+"_max"].values[i] = time_series.max()
        predictors["m"+str(j+1)+"_mean"].values[i] = time_series.mean()
        X = np.array(range(8))
        fit = np.polyfit(X,time_series,2)
        for k in range(3):
            predictors["m"+str(j+1)+"_coef_"+str(k)].values[i] = fit[2-k]


#### Use coeficients of revenue

In [ ]:
for j in range(3):
    predictors["rev_coef_"+str(j)] = np.zeros(predictors.shape[0])
    predictors["rev_max"] = np.zeros(predictors.shape[0])
    #predictors["rev_min"] = np.zeros(predictors.shape[0])
    predictors["rev_mean"] = np.zeros(predictors.shape[0])
    #predictors["rev_std"] = np.zeros(predictors.shape[0])
for i in range(predictors.shape[0]): 
    if predictors["id"].values[i] in revenues["id"].values: 
        curr_rev  = revenues.ix[revenues["id"]== predictors["id"].values[i],:]
        if predictors["device"].values[i] in curr_rev["device"].values:
            curr_rev = curr_rev.ix[curr_rev["device"] == predictors["device"].values[i],:]
            time_series = np.array(curr_rev.ix[:,5:61])[0]
            time_series = [np.log(c) for c in list(time_series) if c>0]
            if len(time_series)<=1: continue
            predictors["rev_max"].values[i] = np.max(time_series)
            predictors["rev_mean"].values[i] = np.mean(time_series)
            X = np.array(range(len(time_series)))
            fit = np.polyfit(X,time_series,2)
            for k in range(3):  predictors["rev_coef_"+str(k)].values[i] = fit[2-k]            

#### Smart Standardization accross the categories


cat_mean = predictors.groupby('category').agg('mean').reset_index()
cat_var = predictors.groupby('category').agg('var').reset_index()
cat_std = cat_mean.merge(cat_var,on = "category",suffixes=('_mean', '_var'))

def smart_standardize(inp):
    temp = predictors[["id","name","category","device",inp]].merge(cat_std[["category",inp+"_mean",inp+"_var"]],how="left",on="category")
    return 1.0*(predictors[inp] - temp[inp+"_mean"])/(temp[inp+"_var"]+1)


'''
for i in range(1,6):
    predictors["star"+str(i)] = smart_standardize("star"+str(i))
   
for j in range(3):
    predictors["rev_coef_"+str(j)]=  smart_standardize("rev_coef_"+str(j))
for i in range(4):
    predictors["m"+str(i+1)+"_max"]  = smart_standardize("m"+str(i+1)+"_max") 
    #predictors["m"+str(i+1)+"_min"]  = smart_standardize("m"+str(i+1)+"_min")
    predictors["m"+str(i+1)+"_mean"]  = smart_standardize("m"+str(i+1)+"_mean")
    #predictors["m"+str(i+1)+"_std"]   = smart_standardize("m"+str(i+1)+"_std")
for j in range(3):
        predictors["m"+str(i+1)+"_coef_"+str(j)]  = smart_standardize("m"+str(i+1)+"_coef_"+str(j))'''

### Take the log


In [ ]:
predictors.head(1)

In [ ]:
#predictors["download_sum"] = np.log(predictors["download_sum"])
#predictors["cumulative_downloads_2015-02"] = np.log(predictors["cumulative_downloads_2015-02"])

## To csv



In [ ]:
predictors.to_csv("predictors.csv")

In [118]:
predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1)
predictors = predictors.fillna(0)

In [ ]:
predictors = predictors.fillna(0)


## How good did the predictor perform   --> Start running from here

In [8]:
#imported libraries
import pandas as pd
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import datetime
%matplotlib inline  
from sklearn import linear_model
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#Creating of the input data
downloads = pd.read_csv('train_app_downloads.csv')
reviews = pd.read_csv('train_app_review.csv')
ratings = pd.read_csv('train_app_rating.csv')
usages = pd.read_csv('train_usage.csv')
revenues = pd.read_csv('train_revenue.csv')
output = pd.read_csv('train_final_downloads.csv')
dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))

#sentiment = pd.read_csv('sentiment.csv',header=-1).ix[:,0]
#sentiment.values[sentiment.values==0.0] = 0.5
#reviews["sentiment_score"] = sentiment.values 

reviews = pd.read_csv('reviews_modified.csv').drop('Unnamed: 0', 1)


#We map -1 to 0 in the downloads (there are no 0 in the initial data)
replacementValue=0
downloads = downloads.replace(-1,replacementValue)

#Minor corrections
ratings = ratings.rename(columns={'start1': 'star1'})
ratings = pd.merge(downloads.drop(dateRange,1), ratings.drop('Unnamed: 0', 1), how='left',
                   on=["id","name","category"]).replace("NaN",replacementValue)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#This is the metric we use to determine our performance
def metric(y_pred,y_test,percent=1):
    top = int(len(y_pred)/100.0*percent)
    return (len(set([i[0] for i in sorted(enumerate(y_pred), key=lambda x:x[1],reverse=True)][0:top])
       .intersection([i[0] for i in sorted(enumerate(y_test), key=lambda x:x[1],reverse=True)][0:top])
               ))/(percent/100.0)/len(y_pred)*100

In [ ]:
predictors = pd.read_csv('predictors_spark.csv',sep='\t').drop('Unnamed: 0', 1).fillna(0)

In [10]:
predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1).fillna(0)

In [ ]:
predictors = predictors.fillna(0)

## Predictor selection 2 methods. Cannot run both...

### Forward Predictor selection

In [ ]:
#Use a model to do a forward recursive predictor selection
#mod=linear_model.Lasso(alpha=100,fit_intercept=False)
#mod=linear_model.LinearRegression(fit_intercept=False)
#mod= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 100) 
#mod= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100) 
mod = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\

rfe = RFE(estimator=mod, n_features_to_select=1, step=1)
rfe.fit(predictors.as_matrix()[:,4:], output.as_matrix()[:,5])
ranking = rfe.ranking_

In [ ]:
for e in [(i[0],i[1]) for i in sorted(zip(predictors.columns[4:],ranking),key=lambda x: x[1])]:
    print e

In [ ]:
from sklearn.feature_selection import RFE, RFECV
mod = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\

rfe_cv = RFECV(estimator=mod, step=1, cv=5)
rfe_cv.fit(predictors.as_matrix()[:,4:], output.as_matrix()[:,5])
ranking = rfe_cv.ranking_

In [ ]:
for e in [(i[0],i[1]) for i in sorted(zip(predictors.columns[4:],ranking),key=lambda x: x[1])]:
    print e

### Import ranking if you do not want to wait for completion

In [ ]:
import pickle

# Saving the objects:
with open('ranking_RFECV.pickle', 'w') as f:
    pickle.dump(ranking, f)

In [ ]:
import pickle
#You can import the rankings directly:
#with open('ranking_RFECV.pickle') as f:
#    ranking = pickle.load(f)
with open('ranking_RFE.pickle') as f:
    ranking = pickle.load(f) 


In [ ]:
#manually drop predictors 
#predictors_to_drop = []
nb_pred_to_keep = 70
predictors_to_drop = [i[0] for i in sorted(zip(predictors.columns[4:],ranking),key=lambda x: x[1])][nb_pred_to_keep:]
for col in predictors_to_drop:
    predictors = predictors.drop(col,1)
    

### Other predictor selection method using Lasso

In [ ]:
# Other method
predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1)
predictors = predictors.fillna(0)
#var_select = linear_model.Lasso(alpha = 0.01).fit(predictor_train_top_10_precent.as_matrix()[:,4:],np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))
var_select = linear_model.Lasso(alpha = 0.1).fit(predictors.as_matrix()[:,4:],np.log(output["cumulative_downloads_2016-02"].as_matrix()))

importance_list = [i for i in range(len(var_select.coef_)) if abs(var_select.coef_[i])>0]
important_predictors = [predictors.columns.values[i+4] for i in range(len(var_select.coef_)) if (i in importance_list)]
print important_predictors
for col in predictors.columns.values[4:]:
    if col not in important_predictors:
        predictors = predictors.drop(col,1)

In [ ]:
predictors = predictors.replace(float(np.inf),0)

# Classification + Regression

In [11]:
print list(predictors.columns.values) 
np.random.seed(1)
K = 5

top_percent_classif = 20


kf = KFold(len(predictors), n_folds=K,shuffle=False)
old_top = []
new_top = []
new_top_noClassif = []
top_10 = []
new_top_select = [] 
new_top_noClassif = [] 
for train, test in kf:
    #base model
    '''old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12], output.as_matrix()[train,5])
    old_y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])
    old_top.append(metric(old_y_pred,output.as_matrix()[test,5]))'''
    
    #model to determine the top 10%   (CLASSIFICATION)
    #mod_class10=GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    mod_class10= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    .fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
    
    y_pred =  mod_class10.predict(predictors.as_matrix()[test,4:])
    
    #estimate of the top 10% of the test set
    estimate_class10 = predictors.iloc[test].copy()
    estimate_class10["firstEstimate"] = y_pred
    estimate_class10 = estimate_class10.sort_values(by= "firstEstimate",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]
    estimate_class10 = estimate_class10.drop("firstEstimate",1)
#     estimate_class10 = estimate_class10.sort_values(by= "daily_avg",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]

    
    #top 10% of the trainning set
    output_train_top_10_precent = output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1)
    predictor_train_top_10_precent = output_train_top_10_precent.merge(predictors, how='left', on=["id","name","category","device"]).copy()
    predictor_train_top_10_precent = predictor_train_top_10_precent.drop('cumulative_downloads_2016-02',1)
    #predictor_train_top_10_precent = predictor_train_top_10_precent.drop('firstEstimate',1)
    
    #This is the actual top 1% of the test set
    output_test_top_1_precent = output.iloc[test].sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(0.01*len(output.iloc[test]))].copy()

    
    #second model -> Regression on the top obtainned by regression
    #mod_top1= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    #mod_top1= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    #mod_top1=linear_model.Lasso(alpha=100,fit_intercept=False)\
    #mod_top1=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')\
    mod_top1 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    .fit(predictor_train_top_10_precent.as_matrix()[:,4:], np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))

    
    y_pred_2 =  mod_top1.predict(estimate_class10.as_matrix()[:,4:])
    
    
    '''#Andrew's regression with additionnal lasso predictor selection
    #(1) feature selection : lasso or random forest
    #var_select = linear_model.Lasso(alpha = 0.01).fit(predictor_train_top_10_precent.as_matrix()[:,4:],np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))
    var_select = RandomForestRegressor(max_features = "sqrt",n_estimators = 100).fit(predictor_train_top_10_precent.as_matrix()[:,4:], np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))
      
    #importance_list = [i for i in  range(len(var_select.coef_)) if abs(var_select.coef_[i])>0]    
    importance_list = list(reversed(np.argsort(var_select.feature_importances_)))[0:20]
    
    #new train and test set with the selected variables
    X_rf_train = predictor_train_top_10_precent.as_matrix()[:,4:][:,importance_list]
    y_rf_train = output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()
    X_rf_test = estimate_class10.as_matrix()[:,4:][:,importance_list]  
    
    #(2) regression: random forest or boosting 
    #mod_top1_select= RandomForestRegressor(max_features = "sqrt",n_estimators = 100).fit(X_rf_train, y_rf_train)
    params = {'n_estimators': 1000, 'max_depth': 2, 'learning_rate': 0.01, 'loss': 'ls'}
    mod_top1_select= GradientBoostingRegressor(**params).fit(X_rf_train, y_rf_train)
    #mod_top1_select = SVR(kernel = 'poly',degree = 3).fit(X_rf_train,y_rf_train)
    y_pred_3 = mod_top1_select.predict(X_rf_test)
    
    #No Classification model
    mod_noClassif= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100).fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
    y_pred_noClassif =  mod_noClassif.predict(predictors.as_matrix()[test,4:])'''

    
    
    
    
    estimate_top1 = estimate_class10.copy()
    #estimate_top1_select = estimate_class10.copy()
    #estimate_top1_noClassif = predictors.iloc[test].copy()
    estimate_top1["secondEstimate"] = y_pred_2
    #estimate_top1_select["thirdEstimate"] = y_pred_3
    #estimate_top1_noClassif["noClassifEstimate"] = y_pred_noClassif
    estimate_top1 = estimate_top1.sort_values(by= "secondEstimate",ascending = False).iloc[:int(0.01*len(output.iloc[test]))]
    
    #estimate_top1_select = estimate_top1_select.sort_values(by= "thirdEstimate",ascending = False).iloc[0:int(0.01*len(output.iloc[test]))]
    #estimate_top1_noClassif = estimate_top1_noClassif.sort_values(by= "noClassifEstimate",ascending = False).iloc[0:int(0.01*len(output.iloc[test]))]

    estimation_error = len(estimate_top1.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent)
    new_top.append(estimation_error)
    print estimation_error
    #new_top_select.append(len(estimate_top1_select.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))
    #new_top_noClassif.append(len(estimate_top1_noClassif.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))
    top_10.append(len(estimate_class10.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))
       
#print "Old model           : " + str(1.0*sum(old_top)/len(old_top))
print "Top10%              : " + str(1.0*sum(top_10)/len(top_10))
print "Top1% with classif1 : " + str(1.0*sum(new_top)/len(new_top))
#print "Top1% with classif2 : " + str(1.0*sum(new_top_select)/len(new_top_select))
#print "Top1% no classif1   : " + str(1.0*sum(new_top_noClassif)/len(new_top_noClassif))

['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'download_sum', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'iphone', 'ipad', 'Productivity', 'Entertainment', 'Travel', 'Sports', 'Music', 'Shopping', 'Finance', 'Business', 'Navigation', 'Food and Drink', 'Utilities', 'Newsstand', 'Health and Fitness', 'News', 'Lifestyle', 'Medical', 'Weather', 'Games', 'Catalogs', 'Social Networking', 'Photo and Video', 'Reference', 'Books', 'Education', 'korean', 'other', 'japanese', 'chinese', 'english', 'cumulative_downloads_2015-02', 'dl_projection', 'avg_review', 'var_review', 'star1', 'star2', 'star3', 'star4', 'star5', 'num_ratings', 'ratings_per_downloads', 'num_versions', 'num_review', 'review_per_downloads', 'days_since_release', 'downloads_per_day_before', 'versions_per_day', 'review_per_day', 'rating_per_day', 'positive', 'negative', 'avg_sentiment_score', 'm1_max', 'm1_m

#70.9375
['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'download_sum', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'iphone', 'ipad', 'Productivity', 'Entertainment', 'Travel', 'Sports', 'Music', 'Shopping', 'Finance', 'Business', 'Navigation', 'Food and Drink', 'Utilities', 'Newsstand', 'Health and Fitness', 'News', 'Lifestyle', 'Medical', 'Weather', 'Games', 'Catalogs', 'Social Networking', 'Photo and Video', 'Reference', 'Books', 'Education', 'korean', 'other', 'japanese', 'chinese', 'english', 'cumulative_downloads_2015-02', 'dl_projection', 'avg_review', 'var_review', 'star1', 'star2', 'star3', 'star4', 'star5', 'num_ratings', 'ratings_per_daily_downloads', 'num_review', 'num_review_per_daily_downloads', 'days_since_release', 'downloads_per_day_before', 'positive', 'negative', 'avg_sentiment_score', 'm1_max', 'm1_min', 'm1_mean', 'm1_std', 'm1_coef_0', 'm1_coef_1', 'm1_coef_2', 'm2_max', 'm2_min', 'm2_mean', 'm2_std', 'm2_coef_0', 'm2_coef_1', 'm2_coef_2', 'm3_max', 'm3_min', 'm3_mean', 'm3_std', 'm3_coef_0', 'm3_coef_1', 'm3_coef_2', 'm4_max', 'm4_min', 'm4_mean', 'm4_std', 'm4_coef_0', 'm4_coef_1', 'm4_coef_2', 'rev_coef_0', 'rev_max', 'rev_min', 'rev_mean', 'rev_std', 'rev_coef_1', 'rev_coef_2', 'num_versions', 'gini_review']
avg_sentiment and positive/negative are 0 seed 5


['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'download_sum', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'Music', 'Shopping', 'Food and Drink', 'Catalogs', 'Social Networking', 'Photo and Video', 'chinese', 'cumulative_downloads_2015-02', 'dl_projection', 'star2', 'star3', 'num_review_per_daily_downloads', 'days_since_release', 'downloads_per_day_before', 'm1_mean', 'm1_coef_1', 'm2_max', 'm2_mean', 'm2_coef_0', 'm2_coef_2', 'm3_coef_0', 'm4_mean', 'rev_coef_0']
Top10%              : 99.6875
Top1% with classif1 : 70.9375
seed 5

### With Weighted regression

In [ ]:
print list(predictors.columns.values) 
np.random.seed(1)
K = 5

top_percent_classif = 12

kf = KFold(len(predictors), n_folds=K)
old_top = []
new_top = []
new_top_noClassif = []
top_10 = []
new_top_select = [] 
new_top_noClassif = [] 
for train, test in kf:
    mod_class10= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    .fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
    
    y_pred =  mod_class10.predict(predictors.as_matrix()[test,4:])
    
    #estimate of the top 10% of the test set
    estimate_class10 = predictors.iloc[test].copy()
    estimate_class10["firstEstimate"] = y_pred
    estimate_class10 = estimate_class10.sort_values(by= "firstEstimate",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]
    estimate_class10 = estimate_class10.drop("firstEstimate",1)
    
    #output_train_top_10_precent = output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1)
    output = output.fillna("No name")
    output_train_top_10_precent = pd.concat([\
 output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).drop('Unnamed: 0',1),\
 output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1),\
 output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1),\
 output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1),\
 output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1),\
 output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1),\
 output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1),\
 output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1)\
 ])
    output_train_top_10_precent = output_train_top_10_precent.drop("name",1)
    predictor_train_top_10_precent = output_train_top_10_precent.merge(predictors, how='left', on=["id","category","device"]).copy()
    predictor_train_top_10_precent = predictor_train_top_10_precent.drop('cumulative_downloads_2016-02',1)
    #predictor_train_top_10_precent = predictor_train_top_10_precent.drop('firstEstimate',1)
    #This is the actual top 1% of the test set
    output_test_top_1_precent = output.iloc[test].sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(0.01*len(output.iloc[test]))].copy()

    
    #second model -> Regression on the top obtainned by regression
    mod_top1 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    .fit(predictor_train_top_10_precent.as_matrix()[:,4:], np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))

    
    y_pred_2 =  mod_top1.predict(estimate_class10.as_matrix()[:,4:])
    
    
    estimate_top1 = estimate_class10.copy()
    estimate_top1["secondEstimate"] = y_pred_2
    estimate_top1 = estimate_top1.sort_values(by= "secondEstimate",ascending = False).iloc[:int(0.01*len(output.iloc[test]))]

    estimation_error = len(estimate_top1.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent)
    new_top.append(estimation_error)
    top_10.append(len(estimate_class10.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))
       
print "Top10%              : " + str(1.0*sum(top_10)/len(top_10))
print "Top1% with classif1 : " + str(1.0*sum(new_top)/len(new_top))


In [ ]:
var_select.feature_importances_

## Cross Validation k-folds

In [111]:


K = 5
print list(predictors.columns.values)
np.random.seed(1)
kf = KFold(len(predictors), n_folds=K,shuffle=True)
new_top = []
for train, test in kf:

    #old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12].astype(float), output['cumulative_downloads_2016-02'].as_matrix()[train])
    #y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])


    #model
    #mod=linear_model.LinearRegression(fit_intercept=False)\ 
    #mod=linear_model.Lasso(alpha=100,fit_intercept=False)\
    #mod = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')\ 
    #mod= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 100)\
    #mod= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    
    mod = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    .fit(predictors.as_matrix()[train,4:].astype(float), output["cumulative_downloads_2016-02"].as_matrix()[train]) 

    y_pred =  mod.predict(predictors.as_matrix()[test,4:].astype(float))
    err = (metric(y_pred,output["cumulative_downloads_2016-02"].as_matrix()[test]))
    new_top.append(err)
    print err

print 1.0*sum(new_top)/len(new_top)

['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'download_sum', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'iphone', 'ipad', 'Productivity', 'Entertainment', 'Travel', 'Sports', 'Music', 'Shopping', 'Finance', 'Business', 'Navigation', 'Food and Drink', 'Utilities', 'Newsstand', 'Health and Fitness', 'News', 'Lifestyle', 'Medical', 'Weather', 'Games', 'Catalogs', 'Social Networking', 'Photo and Video', 'Reference', 'Books', 'Education', 'korean', 'other', 'japanese', 'chinese', 'english', 'cumulative_downloads_2015-02', 'dl_projection', 'avg_review', 'var_review', 'star1', 'star2', 'star3', 'star4', 'star5', 'num_ratings', 'ratings_per_downloads', 'num_versions', 'num_review', 'review_per_downloads', 'days_since_release', 'downloads_per_day_before', 'versions_per_day', 'review_per_day', 'rating_per_day', 'positive', 'negative', 'avg_sentiment_score', 'm1_max', 'm1_m

In [ ]:
print y_pred
print output["cumulative_downloads_2016-02"].as_matrix()[test]

## Attempt at boosting

In [ ]:
print list(predictors.columns.values)
K = 5

for n_est in [1000,5000,10000,50000]:
    for l_rate in [0.001,0.01,0.1]:
        for m_depth in range(1,5):

            np.random.seed(1)
            kf = KFold(len(predictors), n_folds=K)
            old_top = []
            new_top = []
            for train, test in kf:
                #model
                mod = GradientBoostingRegressor(n_estimators=n_est, learning_rate=l_rate,max_depth=m_depth, random_state=0, loss='ls').fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5]) 
                y_pred =  mod.predict(predictors.as_matrix()[test,4:])
                new_top.append(metric(y_pred,output.as_matrix()[test,5]))

            print "Performance with \tB="+str(n_est)+"\tl="+str(l_rate)+"\td="+ str(m_depth)+" is: \t"+str(1.0*sum(new_top)/len(new_top))



## Attempt at SVR

In [ ]:
from sklearn.svm import SVR

print list(predictors.columns.values)
K = 5

for ker in ['linear', 'poly', 'rbf', 'sigmoid']:
    for deg in range(5):
        for coef in [0,1,2]:
            for Cost in [0.001,0.01,0.1,1,10]:
                for epsi in [0.001,0.01,0.1,1,10]:

                    np.random.seed(1)
                    kf = KFold(len(predictors), n_folds=K)
                    old_top = []
                    new_top = []
                    for train, test in kf:
                        #model
                        #mod = SVR(kernel=ker, degree=deg,  coef0=coef, C=Cost, epsilon=epsi).fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5]) 
                        mod = SVR(kernel=ker, degree=deg,  coef0=coef, C=Cost, epsilon=epsi,max_iter=1000).fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5]) 
                        y_pred =  mod.predict(predictors.as_matrix()[test,4:])
                        new_top.append(metric(y_pred,output.as_matrix()[test,5]))

                    print "Performance with \tKernel="+ker+"\tdegree="+str(deg)+"\tcoef0="+ str(coef)+"\tCost="+str(Cost)+"\tepsilon="+ str(epsi)+" is: \t"+str(1.0*sum(new_top)/len(new_top))



# CV on the size of the Classification top

In [ ]:
print list(predictors.columns.values) 

K = 5

cv_top_10 = []
cv_top_1  = []

for top in range(1,31):
    np.random.seed(1)
    top_percent_classif = top

    kf = KFold(len(predictors), n_folds=K)
    old_top = []
    new_top = []
    top_10 = []
    new_top_select = [] 
    for train, test in kf:
        #base model
        #old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12], output.as_matrix()[train,5])
        #old_y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])
        #old_top.append(metric(old_y_pred,output.as_matrix()[test,5]))

        #model to determine the top 10%   (CLASSIFICATION)
        mod_class10= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
        .fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
        y_pred =  mod_class10.predict(predictors.as_matrix()[test,4:])

        #estimate of the top 10% of the test set
        estimate_class10 = predictors.iloc[test].copy()
        estimate_class10["firstEstimate"] = y_pred
        estimate_class10 = estimate_class10.sort_values(by= "firstEstimate",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]
        estimate_class10 = estimate_class10.drop("firstEstimate",1)
        #estimate_class10 = estimate_class10.sort_values(by= "daily_avg",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]


        #top 10% of the trainning set
        output_train_top_10_precent = output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1)
        predictor_train_top_10_precent = output_train_top_10_precent.merge(predictors, how='left', on=["id","name","category","device"]).copy()
        predictor_train_top_10_precent = predictor_train_top_10_precent.drop('cumulative_downloads_2016-02',1)
        #predictor_train_top_10_precent = predictor_train_top_10_precent.drop('firstEstimate',1)

        #This is the actual top 1% of the test set
        output_test_top_1_precent = output.iloc[test].sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(0.01*len(output.iloc[test]))].copy()


        #second model -> Regression on the top obtainned by regression
        mod_top1 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    .fit(predictor_train_top_10_precent.as_matrix()[:,4:], np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))

        y_pred_2 =  mod_top1.predict(estimate_class10.as_matrix()[:,4:])

        estimate_top1 = estimate_class10.copy()
        estimate_top1["secondEstimate"] = y_pred_2

        estimate_top1 = estimate_top1.sort_values(by= "secondEstimate",ascending = False).iloc[0:int(0.01*len(output.iloc[test]))]

        estimation_error = len(estimate_top1.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent)
        new_top.append(estimation_error)
        top_10.append(len(estimate_class10.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))


    cv_top_10.append(1.0*sum(top_10)/len(top_10))
    cv_top_1.append(1.0*sum(new_top)/len(new_top))

plt.plot(range(1,31),cv_top_10,label = 'top 10 estimate')
plt.plot(range(1,31),cv_top_1,label = 'top 1 estimate')
plt.legend(loc = 'best')




# CV on the coefficients of ratings

In [ ]:
predictors['ratings_aggregate'] = 0*predictors['star1'] 

print list(predictors.columns.values) 

K = 5

cv_top_10 = []
cv_top_1  = []

for s1 in range(-5,6):
    for s2 in range(-5,6):
        for s3 in range(-5,6):
            for s4 in range(-5,6):
                for s5 in range(-5,6):
                    predictors = predictors.drop('ratings_aggregate',1)
                    predictors['ratings_aggregate'] = s1*predictors['star1'] +\
                    s2*predictors['star2'] +\
                    s3*predictors['star3'] +\
                    s4*predictors['star4'] +\
                    s5*predictors['star5'] 
                    

                    np.random.seed(1)
                    top_percent_classif = 12

                    kf = KFold(len(predictors), n_folds=K)
                    old_top = []
                    new_top = []
                    top_10 = []
                    new_top_select = [] 
                    for train, test in kf:
                        #base model
                        #old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12], output.as_matrix()[train,5])
                        #old_y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])
                        #old_top.append(metric(old_y_pred,output.as_matrix()[test,5]))

                        #model to determine the top 10%   (CLASSIFICATION)
                        mod_class10= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
                        .fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
                        y_pred =  mod_class10.predict(predictors.as_matrix()[test,4:])

                        #estimate of the top 10% of the test set
                        estimate_class10 = predictors.iloc[test].copy()
                        estimate_class10["firstEstimate"] = y_pred
                        estimate_class10 = estimate_class10.sort_values(by= "firstEstimate",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]
                        estimate_class10 = estimate_class10.drop("firstEstimate",1)
                        #estimate_class10 = estimate_class10.sort_values(by= "daily_avg",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]


                        #top 10% of the trainning set
                        output_train_top_10_precent = output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1)
                        predictor_train_top_10_precent = output_train_top_10_precent.merge(predictors, how='left', on=["id","name","category","device"]).copy()
                        predictor_train_top_10_precent = predictor_train_top_10_precent.drop('cumulative_downloads_2016-02',1)
                        #predictor_train_top_10_precent = predictor_train_top_10_precent.drop('firstEstimate',1)

                        #This is the actual top 1% of the test set
                        output_test_top_1_precent = output.iloc[test].sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(0.01*len(output.iloc[test]))].copy()


                        #second model -> Regression on the top obtainned by regression
                        mod_top1 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
                    .fit(predictor_train_top_10_precent.as_matrix()[:,4:], np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))

                        y_pred_2 =  mod_top1.predict(estimate_class10.as_matrix()[:,4:])

                        estimate_top1 = estimate_class10.copy()
                        estimate_top1["secondEstimate"] = y_pred_2

                        estimate_top1 = estimate_top1.sort_values(by= "secondEstimate",ascending = False).iloc[0:int(0.01*len(output.iloc[test]))]

                        estimation_error = len(estimate_top1.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent)
                        new_top.append(estimation_error)
                        top_10.append(len(estimate_class10.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))


                    cv_top_10.append(1.0*sum(top_10)/len(top_10))
                    cv_top_1.append(1.0*sum(new_top)/len(new_top))
                    print str([s1,s2,s3,s4,s5])+"  "+ str(1.0*sum(new_top)/len(new_top))

#plt.plot(range(0,11),cv_top_10,label = 'top 10 estimate')
#plt.plot(range(0,11),cv_top_1,label = 'top 1 estimate')
#plt.legend(loc = 'best')




# CV on the size of the predictor set

In [ ]:

np.random.seed(1)
K = 5


for nb_pred_to_keep in range(5,100,5):

    predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1)
    predictors = predictors.fillna(0)

    predictors_to_drop = [i[0] for i in sorted(zip(predictors.columns[4:],ranking),key=lambda x: x[1])][nb_pred_to_keep:]
    for col in predictors_to_drop:
        predictors = predictors.drop(col,1)

    kf = KFold(len(predictors), n_folds=K)
    #print list(predictors.columns.values) 
    new_top_noClassif = [] 
    for train, test in kf:

        output_test_top_1_precent = output.iloc[test].sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(0.01*len(output.iloc[test]))].copy()


        #No Classification model
        mod_noClassif= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100).fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
        y_pred_noClassif =  mod_noClassif.predict(predictors.as_matrix()[test,4:])

        estimate_top1_noClassif = predictors.iloc[test].copy()
        estimate_top1_noClassif["noClassifEstimate"] = y_pred_noClassif
        estimate_top1_noClassif = estimate_top1_noClassif.sort_values(by= "noClassifEstimate",ascending = False).iloc[0:int(0.01*len(output.iloc[test]))]

        new_top_noClassif.append(len(estimate_top1_noClassif.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))
        

    print "Top1% no classif with "+str(nb_pred_to_keep)+" kept predictors : " + str(1.0*sum(new_top_noClassif)/len(new_top_noClassif))

RForest pred selection and CV on the number of predictors<br/>['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'iphone', 'ipad', 'Productivity', 'Entertainment', 'Travel', 'Sports', 'Music', 'Shopping', 'Finance', 'Business', 'Navigation', 'Food and Drink', 'Utilities', 'Newsstand', 'Health and Fitness', 'News', 'Lifestyle', 'Medical', 'Weather', 'Games', 'Catalogs', 'Social Networking', 'Photo and Video', 'Reference', 'Books', 'Education', 'chinese', 'Unnamed: 49', 'japanese', 'other', 'english', 'korean', 'cumulative_downloads_2015-02', 'avg_review', 'var_review', 'star1', 'star2', 'star3', 'star4', 'star5', 'num_ratings', 'reviewPerDailyDownloads', 'num_versions', 'num_review', 'days_since_release', 'downloads_per_day_before', 'positive', 'negative', 'm1_min', 'm1_mean', 'm1_std', 'm1_coef_0', 'm1_coef_1', 'm1_coef_2', 'm2_max', 'm2_min', 'm2_coef_0', 'm2_coef_1', 'm2_coef_2', 'm3_min', 'm3_mean', 'm3_std', 'm3_coef_0', 'm3_coef_1', 'm3_coef_2', 'm4_max', 'm4_min', 'm4_mean', 'm4_std', 'm4_coef_0', 'm4_coef_1', 'm4_coef_2', 'rev_coef_0', 'rev_max', 'rev_min', 'rev_std', 'rev_coef_1', 'rev_coef_2'] <br/>
Top1% no classif with 5 kept predictors : 62.5396825397    <br/>
Top1% no classif with 10 kept predictors : 65.0793650794<br/>
Top1% no classif with 15 kept predictors : 65.3968253968<br/>
Top1% no classif with 20 kept predictors : 64.7619047619<br/>
Top1% no classif with 25 kept predictors : 64.7619047619<br/>
Top1% no classif with 30 kept predictors : 65.3968253968<br/>
Top1% no classif with 35 kept predictors : 66.0317460317<br/>
Top1% no classif with 40 kept predictors : 65.7142857143<br/>
Top1% no classif with 45 kept predictors : 65.0793650794<br/>
Top1% no classif with 50 kept predictors : 65.3968253968<br/>
Top1% no classif with 55 kept predictors : 65.0793650794<br/>
Top1% no classif with 60 kept predictors : 65.3968253968<br/>
Top1% no classif with 65 kept predictors : 64.4444444444<br/>
Top1% no classif with 70 kept predictors : 65.3968253968<br/>
Top1% no classif with 75 kept predictors : 63.8095238095<br/>
Top1% no classif with 80 kept predictors : 63.4920634921<br/>
Top1% no classif with 85 kept predictors : 64.4444444444<br/>
Top1% no classif with 90 kept predictors : 61.9047619048<br/>



.
.
.
.

# Old Testing model

In [ ]:
old_top = []
new_top = []

test_frac = 0.31  #Fraction of test points
N = 20   #number of iterations
np.random.seed(1)
for i in range(1,N):
    r = np.random.randint(1,429496729)
    X_train, X_test, y_train, y_test = train_test_split(predictors.as_matrix()[:,4:12], output.as_matrix()[:,5], test_size=test_frac, random_state=r)
    X_test = X_test[0:10000]
    y_test = y_test[0:10000]
    old_mod=linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train)
    old_y_pred =  old_mod.predict(X_test)
    old_top.append(metric(old_y_pred,y_test))
    
np.random.seed(1)
for i in range(1,N):
    r = np.random.randint(1,429496729)
    X_train, X_test, y_train, y_test = train_test_split(predictors.as_matrix()[:,4:], output.as_matrix()[:,5], test_size=test_frac, random_state=r)
    X_test = X_test[0:10000]
    y_test = y_test[0:10000]
    #mod=linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train)
    #mod=linear_model.Lasso(alpha=100,fit_intercept=False).fit(X_train,y_train)
    #mod= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 100).fit(X_train,y_train)
    mod= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100).fit(X_train,y_train)
    y_pred =  mod.predict(X_test)
    new_top.append(metric(y_pred,y_test))
if (N<300):
    fig = plt.figure()
    plt.plot(range(1,N),old_top,label="old")
    plt.plot(range(1,N),new_top,label="performance")
    plt.legend(loc='best')
    plt.show()

print 1.0*sum(old_top)/len(old_top)
print list(predictors.columns.values)
print 1.0*sum(new_top)/len(new_top)

54.8947368421  
55.0526315789  #with #missing and Lasso <br/>
54.7894736842  #including raw ratings Lasso :( <br/>
55.0526315789  #with #missing weightedSumRatings and Lasso <br/>
55.1052631579  #adding the categories <br/>
55.3684210526  #adding average sentiment score and positive/negative label <br/>
55.5263157895  #adding coeficients of usages


With 10000 only:
54.8947368421 all

56.5263157895 on Lasso all but 'rev_coef_i'

Lasso All alpha = 100
['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'iphone', 'ipad', 'Productivity', 'Entertainment', 'Travel', 'Sports', 'Music', 'Shopping', 'Finance', 'Business', 'Navigation', 'Food and Drink', 'Utilities', 'Newsstand', 'Health and Fitness', 'News', 'Lifestyle', 'Medical', 'Weather', 'Games', 'Catalogs', 'Social Networking', 'Photo and Video', 'Reference', 'Books', 'Education', 'avg_review', 'var_review', 'star1', 'star2', 'star3', 'star4', 'star5', 'positive', 'negative', 'm1_coef_0', 'm1_coef_1', 'm1_coef_2', 'm2_coef_0', 'm2_coef_1', 'm2_coef_2', 'm3_coef_0', 'm3_coef_1', 'm3_coef_2', 'm4_coef_0', 'm4_coef_1', 'm4_coef_2']
57.1052631579

Predictor selection, pick the top 50 features selected in the forward recursive selection with Lasso and then do random forest

54.6842105263
['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'iphone', 'ipad', 'Productivity', 'Entertainment', 'Sports', 'Music', 'Shopping', 'Finance', 'Business', 'Navigation', 'Food and Drink', 'Utilities', 'News', 'Lifestyle', 'Medical', 'Weather', 'Games', 'Catalogs', 'Social Networking', 'Photo and Video', 'Reference', 'Education', 'avg_review', 'var_review', 'star2', 'star3', 'star4', 'positive', 'negative', 'm2_coef_0', 'm3_coef_0', 'm3_coef_1', 'm4_coef_2']
57.3157894737

In [ ]:
#Estimation of alpha for the Lasso regression "cros validation" like approach
for a in np.arange(90,110,5):

    old_top = []
    new_top = []

    test_frac = 0.31  #Fraction of test points
    N = 20   #number of iterations


    np.random.seed(1)
    for i in range(1,N):
        r = np.random.randint(1,429496729)
        X_train, X_test, y_train, y_test = train_test_split(predictors.as_matrix()[:,4:], output.as_matrix()[:,5], test_size=test_frac, random_state=r)
        X_test = X_test[0:10000]
        y_test = y_test[0:10000]
        mod=linear_model.Lasso(alpha=a,fit_intercept=False).fit(X_train,y_train) 
        y_pred =  mod.predict(X_test)
        new_top.append(metric(y_pred,y_test))


    print a
    print 1.0*sum(new_top)/len(new_top)
    print " "

# Making Classification on top 10%

In [ ]:
predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1)
predictors = predictors.fillna(0)

thr = predictors.sort_values(by= 'week_8',ascending = False).iloc[int(0.1*len(predictors))]["week_8"]
predictors  = predictors[predictors.week_8 >= thr]


len(predictors)

In [ ]:
output_classification_10 = output
output_classification_50 = output
output_classification_100 = output
#Is top ?
def generator_istop(inp,threshold):
    if (inp[5] >= threshold):
        return 'True'
    else:
        return 'False'

threshold = output.sort_values('cumulative_downloads_2016-02',ascending = False).iloc[len(output)/10]['cumulative_downloads_2016-02']
output_classification_10["is_top"] = output.apply(generator_istop,axis = 1,args=(threshold,))

threshold = output.sort_values('cumulative_downloads_2016-02',ascending = False).iloc[len(output)/50]['cumulative_downloads_2016-02']
output_classification_50["is_top"] = output.apply(generator_istop,axis = 1,args=(threshold,))

threshold = output.sort_values('cumulative_downloads_2016-02',ascending = False).iloc[len(output)/100]['cumulative_downloads_2016-02']
output_classification_100["is_top"] = output.apply(generator_istop,axis = 1,args=(threshold,))

In [ ]:
output_classification_10.head()

In [ ]:
from sklearn.svm import SVC
K=10
kf = KFold(32339, n_folds=K)
old_top = []
new_top = []
for train, test in kf:
    #model
    mod = SVC(C=10.0, kernel='poly', degree=3, gamma='auto', coef0=2.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=10, decision_function_shape=None, random_state=None).fit(predictors.as_matrix()[train,4:],output_classification_10.as_matrix()[train,6])
    #mod = linear_model.LogisticRegression().fit(predictors.as_matrix()[train,4:], output_classification_10.as_matrix()[train,6])
    y_pred =  mod.predict(predictors.as_matrix()[test,4:])
    new_top.append(metric_classification(y_pred,output_classification_10.as_matrix()[test,6]))
print list(predictors.columns.values)
print 1.0*sum(new_top)/len(new_top)


In [ ]:
###This metric is not very good, put True everywhere and get 100%
def metric_classification(y_pred,y_test):
    nb_top = 0
    for i in range(len(y_test)):
        if (y_pred[i]=='True' and y_test[i]=="False"):
            nb_top+=1
    return 100.0*nb_top/len(y_pred)


In [ ]:
list(y_pred).count("False")*1.0/len(y_pred)

In [ ]:
print list(y_pred).count('True')
print list(output_classification_10.as_matrix()[test,6]).count('True')
print len(y_pred)

Question, what if instead of using the 'cumulative_downloads_2016-02' we used exp('cumulative_downloads_2016-02') to try to shrunk the lowest points?


In [ ]:
output_scaled = output
#Is top ?
def generator_outputScaler(inp):
    #return math.exp(inp[5]*1.0/12329752*100)   #28%
    #return 1.0/(20000000-inp[5])   #0.618429189858
    #return inp[5]**2       #53.8045828797
    #return (inp[5]*1.0/20000000)**2*inp[5]     #46.3834326015
    #return (inp[5]*1.0/1500000)**0.5   #  56.5875142341
    #return 100.0/(1+math.exp(-(inp[5]-1500000)*1.0/100000))    #54.4230120696
    #return 200.0/(1+math.exp(-(inp[5]-1500000)*1.0/1000000)) - 100    #55.9691806875
    #return math.log(inp[5])      #58.7522076851
    return inp[5]               #58.4429930902
    #return 200.0/(1+math.exp(-(inp[5]*1.0/1500000-1)*10)) - 100       #55.3506558544
    #return math.exp(1.0/(1+math.exp(-(inp[5]*1.0/1500000-1)*10)))     #


output_scaled["scaled_downloads"] = output.apply(generator_outputScaler,axis = 1)

#output_scaled["scaled_downloads"] =(output['cumulative_downloads_2016-02']-output['cumulative_downloads_2016-02'].mean())/output['cumulative_downloads_2016-02'].std()

In [ ]:
output_scaled.sort_values(by= 'cumulative_downloads_2016-02',ascending = False).head(320)

In [ ]:
plt.plot(list(output_scaled.sort_values(by= 'cumulative_downloads_2016-02',ascending = False)["scaled_downloads"]))
plt.axis([0, 300,min(list(output_scaled.sort_values(by= 'cumulative_downloads_2016-02',ascending = False)["scaled_downloads"])),max(list(output_scaled.sort_values(by= 'cumulative_downloads_2016-02',ascending = False)["scaled_downloads"]))])

In [ ]:
K = 10

kf = KFold(32339, n_folds=K)
old_top = []
new_top = []
for train, test in kf:
    #base model
    old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12], output.as_matrix()[train,5])
    old_y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])
    old_top.append(metric(old_y_pred,output.as_matrix()[test,5]))
    #model
    #mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:], output_scaled.as_matrix()[train,6])
    #mod=linear_model.Lasso(alpha=100,fit_intercept=False).fit(predictors.as_matrix()[train,4:], output_scaled.as_matrix()[train,6])
    #mod = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls').fit(predictors.as_matrix()[train,4:], output_scaled.as_matrix()[train,6]) 
    #mod= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 100).fit(predictors.as_matrix()[train,4:], output_scaled.as_matrix()[train,6])
    mod= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100).fit(predictors.as_matrix()[train,4:], output_scaled.as_matrix()[train,6])
    y_pred =  mod.predict(predictors.as_matrix()[test,4:])
    new_top.append(metric(y_pred,output_scaled.as_matrix()[test,6]))
print 1.0*sum(old_top)/len(old_top)
print list(predictors.columns.values)
print 1.0*sum(new_top)/len(new_top)

In [ ]:
plt.plot(list(output.sort_values(by= 'cumulative_downloads_2016-02',ascending = False)["cumulative_downloads_2016-02"]))
plt.axis([0, 3200, 0, 13000000])

In [ ]:
plt.plot(list(output_scaled.sort_values(by= 'cumulative_downloads_2016-02',ascending = False)["scaled_downloads"]))


In [ ]:
list(set(lang_series))

In [ ]:
predictors

In [ ]:
predictors.sort_values(by= 'week_8',ascending = False)[0:3233].head(3)

In [ ]:
output.sort_values(by= 'cumulative_downloads_2016-02',ascending = False)[0:323].head(3)

In [ ]:
len(predictors.sort_values(by= 'week_8',ascending = False)[0:3233].merge(output.sort_values(by= 'cumulative_downloads_2016-02',ascending = False)[0:323], how='inner', on=["id","name","category","device"]))

In [ ]:
predictors = predictors[:30]
downloads = downloads[:30]
reviews = reviews[:300]
ratings = ratings[:30]
usages = usages[:30]
revenues = revenues[:30]
output =  output[:30]
dateRange = dateRange[:30]
prev_downloads = prev_downloads[:30]

In [ ]:
def detect_language(x):
    try:
        detected = langdetect.detect(x.decode('utf8','ignore'))
        if detected in ['ja']:
            return 'japanese'
        elif detected in ['zh-cn']:
            return 'chinese'
        elif detected in ['ko']:
            return 'korean'
        elif detected in ['en']:
            return 'english'
        return 'other'
    except:
        return 'other'
        #return None

def set_lang_categories(x, cat):
    if x == cat:
        return 1
    else:
        return 0

reviews_withLang = reviews.copy()
reviews_lang = reviews['text'].apply(detect_language)
for cat in list(set(reviews_lang)):
    reviews_withLang[cat] = reviews_lang.apply(set_lang_categories, args=(cat,))
    
#lang_series = predictors['name'].apply(detect_language)
#for cat in list(set(lang_series)):
#    predictors[cat] = lang_series.apply(set_lang_categories, args=(cat,))

reviews_withLang

In [ ]:
reviews_withLang = reviews.copy()
reviews_lang = reviews['text'].apply(detect_language)
for cat in list(set(reviews_lang)):
    reviews_withLang[cat] = reviews_lang.apply(set_lang_categories, args=(cat,))

reviews_withLang = reviews_withLang.groupby('id')[list(set(reviews_lang))].sum().reset_index()

def gini_impurity(inp):
    tot = sum(inp[1:])
    return sum([1.0*x / tot * (1 - 1.0*x / tot) for x in inp[1:]])

reviews_withLang["gini_reviews"] = reviews_withLang.apply(gini_impurity,axis=1)
predictors["gini_reviews"] = predictors.join(reviews_withLang["gini_reviews"],on='id')["gini_reviews"].replace("NaN",0)

In [ ]:
predictors 

In [ ]:
reviews_withLang

In [ ]:



estimate_top1 = estimate_class10.copy()
estimate_top1["secondEstimate"] = np.exp(y_pred_2).astype(int)
estimate_top1 = estimate_top1.merge(output, how='left', on=["id","name","category","device"]).copy()
estimate_top1 = estimate_top1.drop("Unnamed: 0",1)
estimate_top1.sort_values(by= 'cumulative_downloads_2016-02',ascending = False)


In [ ]:
output_test_top_1_precent.join(estimate_class10.ix[:,['days_since_release','cumulative_downloads_2015-02','week_1','week_4','week_8','download_sum']]).sort_values('cumulative_downloads_2016-02',ascending=False)[~output_test_top_1_precent.index.isin(estimate_top1_select.index)]

In [ ]:
predictors[["name","gini_review"]]

In [ ]:
sum_reviews[sum_reviews.name=="AIM: Chat, Free Text, Photo Share, Voice Message"]

In [ ]:
predictors['num_review_per_daily_downloads']

In [ ]:
reviews_old[reviews_old.id == 284776127]

In [ ]:
predictors["num_versions"]

In [ ]:
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('ranking_RFECV.pickle', 'w') as f:
    pickle.dump(ranking, f)

In [ ]:
predictors.head(1)

In [ ]:
mod=linear_model.LinearRegression(fit_intercept=True)\
.fit(predictors[["download_sum","cumulative_downloads_2015-02"]].as_matrix(),output[["cumulative_downloads_2016-02"]].as_matrix())

print mod.coef_

In [ ]:
plt.plot(range(1,31),cv_top_10,label = 'top 10 estimate')
plt.plot(range(1,31),cv_top_1,label = 'top 1 estimate')
plt.legend(loc = 'best')
cv_top_1

In [ ]:
predictors

In [ ]:
range(-5,5)

In [ ]:
predictors = predictors.iloc[np.random.permutation(len(predictors))]

In [ ]:
predictors = pd.read_csv('../predictors_71.5.csv').drop("Unnamed: 0",1)

In [ ]:
downloads

In [ ]:
reviews["name.1"].unique()

In [ ]:
reviews_modified = pd.read_csv('reviews_modified.csv').drop('Unnamed: 0', 1)
reviews_modified.columns.values

In [ ]:
reviews

In [ ]:
from sklearn import datasets
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve,auc
from statsmodels.tools import categorical
import numpy as np

iris = datasets.load_iris()
# Use only data for 2 classes.
X = iris.data[(iris.target==0) | (iris.target==1)]
Y = iris.target[(iris.target==0) | (iris.target==1)]

# Class 0 has indices 0-49. Class 1 has indices 50-99.
# Divide data into 80% training, 20% testing.
train_indices = list(range(40)) + list(range(50,90))
test_indices = list(range(40,50)) + list(range(90,100))
X_train = X[train_indices]
X_test = X[test_indices]
y_train = Y[train_indices]
y_test = Y[test_indices]


###########################################################################
###### Convert categorical variable to matrix and merge back with training
###### data.

# Fake categorical variable.
catVar = np.array(['a']*40 + ['b']*40)
catVar = categorical(catVar, drop=True)
X_train = np.concatenate((X_train, catVar), axis = 1)

catVar = np.array(['a']*10 + ['b']*10)
catVar = categorical(catVar, drop=True)
X_test = np.concatenate((X_test, catVar), axis = 1)
###########################################################################

# Model and test.
clf = GradientBoostingClassifier(learning_rate=0.01,max_depth=8,n_estimators=50).fit(X_train, y_train)

prob = clf.predict_proba(X_test)[:,1]   # Only look at P(y==1).

fpr, tpr, thresholds = roc_curve(y_test, prob)
roc_auc_prob = auc(fpr, tpr)

print(prob)
print(y_test)
print(roc_auc_prob)

In [ ]:
on = 'week_8'

plt.scatter(predictors[on],output['cumulative_downloads_2016-02']\
,c = 2*(output['cumulative_downloads_2016-02']>sorted(output['cumulative_downloads_2016-02'])\
            [int(len(output['cumulative_downloads_2016-02'])*0.99)])\
              +(predictors[on]> sorted(predictors[on])[int(len(output['cumulative_downloads_2016-02'])*0.88)]))
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [172]:
predictors

,id,name,category,device,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,daily_avg,download_sum,coef_0,coef_1,coef_2,coef_3,maxStep,minStep,std,nb_missing,iphone,ipad,Productivity,Entertainment,Travel,Sports,Music,Shopping,Finance,Business,Navigation,Food and Drink,Utilities,Newsstand,Health and Fitness,News,Lifestyle,Medical,Weather,Games,Catalogs,Social Networking,Photo and Video,Reference,Books,Education,korean,other,japanese,chinese,english,cumulative_downloads_2015-02,dl_projection,avg_review,var_review,star1,star2,star3,star4,star5,num_ratings,ratings_per_downloads,num_versions,num_review,review_per_downloads,days_since_release,downloads_per_day_before,versions_per_day,review_per_day,rating_per_day,positive,negative,avg_sentiment_score,m1_max,m1_mean,m1_coef_0,m1_coef_1,m1_coef_2,m2_max,m2_mean,m2_coef_0,m2_coef_1,m2_coef_2,m3_max,m3_mean,m3_coef_0,m3_coef_1,m3_coef_2,m4_max,m4_mean,m4_coef_0,m4_coef_1,m4_coef_2,rev_coef_0,rev_max,rev_mean,rev_coef_1,rev_coef_2
0,281704574,"AIM: Chat, Free Text, Photo Share, Voice Message",Social Networking,iphone,6.239161,6.127804,6.101279,6.183265,6.078625,5.926926,5.792578,6.059791,433.678571,24286,13035.250000,428.710663,-1.520371,-0.013710,308,-233,91.952066,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,617121,787123,3.200000,2.684058,0.189152,0.080590,0.145992,0.139242,0.445023,469040.0,0.760044,3.0,70.0,0.000113,2424.0,254.482887,0.001208,0.028180,188.824477,0,1,3.272679e-01,0.035065,0.032192,0.032400,-0.001056,0.000199,0.216173,0.212028,0.214854,-0.000385,-8.450000e-05,16.729371,15.190247,15.083152,-0.472173,0.100554,96931.0,88379.625,88130.375000,-2675.898810,549.422619,0.000000,0.0,0.000000,0.000000,0.000000
1,281922769,Mobile MIM,Medical,ipad,4.075113,4.084775,3.964886,4.198275,4.089571,4.027899,4.124828,3.991626,58.678571,3286,1679.696429,59.190670,0.011402,-0.002227,28,-34,8.860239,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,83281,106283,1.000000,0.000000,0.256757,0.131532,0.172072,0.142342,0.296396,1109.0,0.013316,1.0,1.0,0.000012,2424.0,34.342680,0.000403,0.000403,0.446457,0,1,3.765728e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,281922769,Mobile MIM,Medical,iphone,4.576183,4.532599,4.418841,4.507715,4.480578,4.464265,4.439284,4.552121,89.851852,4852,2508.285714,85.458031,-0.046052,-0.000866,31,-20,19.594486,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,99000,132964,1.000000,0.000000,0.256757,0.131532,0.172072,0.142342,0.296396,1109.0,0.011202,1.0,1.0,0.000010,2424.0,40.824742,0.000403,0.000403,0.446457,0,1,3.765728e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,281935788,Epocrates References & Tools for Healthcare Pr...,Medical,ipad,5.617810,5.557379,5.520317,5.371302,5.493649,5.418637,5.434969,5.393628,239.660714,13421,7074.410714,237.172215,-0.428057,0.006969,120,-73,31.992341,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,291496,385443,2.666667,2.941176,0.281447,0.103090,0.140426,0.140326,0.334690,50353.0,0.172739,3.0,18.0,0.000062,2424.0,120.204536,0.001208,0.007246,20.270934,0,1,2.368771e-01,0.133475,0.126261,0.128980,-0.000505,-0.000054,0.642443,0.637267,0.630600,0.004964,-6.119286e-04,20.860287,19.811508,20.453041,-0.239506,0.011242,363825.0,346531.750,350810.583333,-516.928571,-141.119048,415.328688,1708.0,359.142857,-0.732351,-0.035426
4,281935788,Epocrates References & Tools for Healthcare Pr...,Medical,iphone,7.038282,7.073875,7.173192,7.016097,7.020318,7.022613,7.028075,7.001246,1150.660714,64437,33222.107143,1157.122078,-1.299754,-0.023838,394,-276,154.055797,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,969059,14201

In [7]:
print list(predictors.columns.values) 
np.random.seed(1)
K = 5

top_percent_classif = 20


kf = KFold(len(predictors), n_folds=K,shuffle=False)
old_top = []
new_top = []
new_top_noClassif = []
top_10 = []
new_top_select = [] 
new_top_noClassif = [] 
for train, test in kf:
    #base model
    '''old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12], output.as_matrix()[train,5])
    old_y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])
    old_top.append(metric(old_y_pred,output.as_matrix()[test,5]))'''
    
    #model to determine the top 10%   (CLASSIFICATION)
    #mod_class10=GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    mod_class10= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100,random_state=0)\
    .fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
    
    y_pred =  mod_class10.predict(predictors.as_matrix()[test,4:])
    
    #estimate of the top 10% of the test set
    estimate_class10 = predictors.iloc[test].copy()
    estimate_class10["firstEstimate"] = y_pred
    estimate_class10 = estimate_class10.sort_values(by= "firstEstimate",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]
    estimate_class10 = estimate_class10.drop("firstEstimate",1)
#     estimate_class10 = estimate_class10.sort_values(by= "daily_avg",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]

    
    #top 10% of the trainning set
    output_train_top_10_precent = output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1)
    predictor_train_top_10_precent = output_train_top_10_precent.merge(predictors, how='left', on=["id","name","category","device"]).copy()
    predictor_train_top_10_precent = predictor_train_top_10_precent.drop('cumulative_downloads_2016-02',1)
    #predictor_train_top_10_precent = predictor_train_top_10_precent.drop('firstEstimate',1)
    
    #This is the actual top 1% of the test set
    output_test_top_1_precent = output.iloc[test].sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(0.01*len(output.iloc[test]))].copy()

    threshold_20 = sorted(output.as_matrix()[train,5],reverse = True)[int(len(train)/5)]
    threshold_10 = sorted(output.as_matrix()[train,5],reverse = True)[int(len(train)/10)]
    threshold_5 = sorted(output.as_matrix()[train,5],reverse = True)[int(len(train)/20)]
    threshold_1 = sorted(output.as_matrix()[train,5],reverse = True)[int(len(train)/20)]
    threshold_0 = 0
    
    relevance =(range(len(train)))
    relevance.sort(key=lambda x: (output.as_matrix()[train,5][x]))
    relevance = (np.asarray(relevance)-int(99.0*len(train)/100))
    relevance[relevance<0] = 0
    
    weights = (output.as_matrix()[train,5] >  threshold_0).astype(int)\
    +0*(output.as_matrix()[train,5] >  threshold_20).astype(int)\
    +0*(output.as_matrix()[train,5] >  threshold_10).astype(int)\
    +0*(output.as_matrix()[train,5] >  threshold_5).astype(int)\
    + 0*(output.as_matrix()[train,5] >  threshold_1).astype(int)\
    + relevance
        
    
    #second model -> Regression on the top obtainned by regression
    #mod_top1= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    #mod_top1= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    #mod_top1=linear_model.Lasso(alpha=100,fit_intercept=False)\
    #mod_top1=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')\
    mod_top1 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    .fit(predictors.as_matrix()[train,4:],np.log(output["cumulative_downloads_2016-02"].as_matrix()[train]) ,sample_weight \
    =(weights)*1.0/sum(weights))
    

    
    y_pred_2 =  mod_top1.predict(estimate_class10.as_matrix()[:,4:])
    
    
    '''#Andrew's regression with additionnal lasso predictor selection
    #(1) feature selection : lasso or random forest
    #var_select = linear_model.Lasso(alpha = 0.01).fit(predictor_train_top_10_precent.as_matrix()[:,4:],np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))
    var_select = RandomForestRegressor(max_features = "sqrt",n_estimators = 100).fit(predictor_train_top_10_precent.as_matrix()[:,4:], np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))
      
    #importance_list = [i for i in  range(len(var_select.coef_)) if abs(var_select.coef_[i])>0]    
    importance_list = list(reversed(np.argsort(var_select.feature_importances_)))[0:20]
    
    #new train and test set with the selected variables
    X_rf_train = predictor_train_top_10_precent.as_matrix()[:,4:][:,importance_list]
    y_rf_train = output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()
    X_rf_test = estimate_class10.as_matrix()[:,4:][:,importance_list]  
    
    #(2) regression: random forest or boosting 
    #mod_top1_select= RandomForestRegressor(max_features = "sqrt",n_estimators = 100).fit(X_rf_train, y_rf_train)
    params = {'n_estimators': 1000, 'max_depth': 2, 'learning_rate': 0.01, 'loss': 'ls'}
    mod_top1_select= GradientBoostingRegressor(**params).fit(X_rf_train, y_rf_train)
    #mod_top1_select = SVR(kernel = 'poly',degree = 3).fit(X_rf_train,y_rf_train)
    y_pred_3 = mod_top1_select.predict(X_rf_test)
    
    #No Classification model
    mod_noClassif= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100).fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
    y_pred_noClassif =  mod_noClassif.predict(predictors.as_matrix()[test,4:])'''

    
    
    
    
    estimate_top1 = estimate_class10.copy()
    #estimate_top1_select = estimate_class10.copy()
    #estimate_top1_noClassif = predictors.iloc[test].copy()
    estimate_top1["secondEstimate"] = y_pred_2
    #estimate_top1_select["thirdEstimate"] = y_pred_3
    #estimate_top1_noClassif["noClassifEstimate"] = y_pred_noClassif
    estimate_top1 = estimate_top1.sort_values(by= "secondEstimate",ascending = False).iloc[:int(0.01*len(output.iloc[test]))]
    
    #estimate_top1_select = estimate_top1_select.sort_values(by= "thirdEstimate",ascending = False).iloc[0:int(0.01*len(output.iloc[test]))]
    #estimate_top1_noClassif = estimate_top1_noClassif.sort_values(by= "noClassifEstimate",ascending = False).iloc[0:int(0.01*len(output.iloc[test]))]

    estimation_error = len(estimate_top1.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent)
    new_top.append(estimation_error)
    print estimation_error
    #new_top_select.append(len(estimate_top1_select.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))
    #new_top_noClassif.append(len(estimate_top1_noClassif.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))
    top_10.append(len(estimate_class10.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))
       
#print "Old model           : " + str(1.0*sum(old_top)/len(old_top))
print "Top10%              : " + str(1.0*sum(top_10)/len(top_10))
print "Top1% with classif1 : " + str(1.0*sum(new_top)/len(new_top))
#print "Top1% with classif2 : " + str(1.0*sum(new_top_select)/len(new_top_select))
#print "Top1% no classif1   : " + str(1.0*sum(new_top_noClassif)/len(new_top_noClassif))

NameError: name 'predictors' is not defined

In [6]:
pd.read_csv("train_rating_max_country.csv")

,Unnamed: 0,id,max_country,star1,star2,star3,star4,star5
0,0,281704574,US,84127,35652,65125,62839,204464
1,1,281922769,US,141,87,119,113,231
2,2,281935788,US,12815,4714,6538,6668,16244
3,3,284147312,US,194,122,166,308,779
4,4,284574017,US,7913,4570,8599,7152,10223
5,5,284735786,US,84292,14481,8013,3238,10500
6,6,284776127,US,2581,1536,1727,1491,2122
7,7,284803487,US,12848,5280,3661,1758,2930
8,8,284806204,US,13315,6784,8564,8045,19258
9,9,284809790,US,16486,6904,9201,8860,22676


In [3]:
rrr = pd.read_csv("train_rating_by_country.csv")

In [5]:
rrr

,Unnamed: 0,id,max_country,star1,star2,star3,star4,star5
0,0,281704574,US,84127,35652,65125,62839,204464
1,1,281704574,JP,75,45,79,57,148
2,2,281704574,GB,2276,909,1326,849,1782
3,3,281704574,SG,52,11,21,25,61
4,4,281704574,DE,1533,889,1542,1262,1611
5,5,281704574,SE,81,49,53,41,66
6,6,281704574,IN,35,16,20,20,60
7,7,281704574,KR,44,22,34,10,61
8,8,281704574,AU,497,207,276,207,481
9,9,281922769,JP,19,12,12,11,12


In [ ]:
Baseline
64.0625
65.625
64.0625
59.375
67.1875
Top10%              : 99.375
Top1% with classif1 : 64.0625
    
Attempt 0 1 5 10 20 same as only 0
62.5
68.75
65.625
60.9375
67.1875
Top10%              : 99.375
Top1% with classif1 : 65.0
    
Attempt 1 * 20
62.5
64.0625
60.9375
59.375
65.625
Top10%              : 99.375
Top1% with classif1 : 62.5
    
Attempt 20 *1
64.0625
67.1875
64.0625
62.5
67.1875
Top10%              : 99.375
Top1% with classif1 : 65.0